In [ ]:
%gui asyncio
import io
import math
import queue
import asyncio
import functools
import threading
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import time
import re
import cv2
import serial
import sys
import glob
import numpy as np
import PIL
import pandas as pd
import plotly.graph_objects as go
from PIL import Image

In [ ]:
##================CAMERA STREAM CLASS================##
class CameraStream:
    captureObject=None
    def __init__(self):
        self.captureObject = cv2.VideoCapture(0)
    def get_frame(self):
        ret, img = self.captureObject.read()
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return self.image_to_byte_array(PIL.Image.fromarray(rgb))
    #https://stackoverflow.com/questions/2659312/how-do-i-convert-a-numpy-array-to-and-display-an-image
    def image_to_byte_array(self, image:Image):
      imgByteArr = io.BytesIO()
      image.save(imgByteArr, format='PNG')
      imgByteArr = imgByteArr.getvalue()
      return imgByteArr
##================SERIAL UTILITY FUNCTIONS================##
#https://stackoverflow.com/questions/12090503/listing-available-com-ports-with-python
def get_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result
def list_ports():
    print('Available serial ports: ')
    ports=get_ports()
    for port in ports:
        print(ports.index(port), end=':')
        print(port)
def try_connect(portindex):
    myport = None
    try:
        myport = serial.Serial(
            port=get_ports()[portindex],
            baudrate=250000,
            timeout=0,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS
        )
        print('Sucessfully connected to port '+myport.port)
        return myport
    except (OSError, serial.SerialException):
        print('ERROR: ', end='')
        if myport.is_open:
            print('Port already open')
        elif myport.port not in portlist:
            print('Port '+myport.port+' not found')
        else:
            print('Could not connect to port')
        pass
##================SURFACE PLOT MATRIX================##
def create_plot_data():
    gridsize = [gridend[0]-gridstart[0], gridend[1]-gridstart[1]]
    gridpoints = [math.floor(gridsize[0]/gridres)+1, math.floor(gridsize[1]/gridres)+1]
    x, y = np.linspace(0, gridsize[0], gridpoints[0]), np.linspace(0, gridsize[1], gridpoints[1])
    z = np.zeros((gridpoints[1], gridpoints[0]))
    return (x, y, z)

In [ ]:
## CONSTANTS ##
CMDPROMPT='>>: '
LINETERM='\r\n'
currentpos=[0,0,0]
gridstart=[0,0,0]
gridend=[50, 100,0]
gridres=10
myport = None
got_ok = True
stream = CameraStream()
tasq = queue.Queue(0) #big brain
plotdata = create_plot_data()

In [ ]:
def read_serial():
    return myport.readline().decode().rstrip()
def contains_ok(line):
    if line == 'ok':
        return True
    return False
def handle_input(sender):
    userinput=sender.value
    sender.value=''
    if userinput in macros:
        terminalout.append_display_data("MACRO: "+userinput)
        macros[userinput]()
    else:
        send_gcode(userinput)
def send_gcode(gcode=''):
    raw = gcode.upper()
    machineout.append_display_data('SEND: '+raw)
    sInput = (raw + LINETERM) #append terminator string
    bInput = sInput.encode() #convert string to bytes
    myport.write(bInput) #send the command
def current_milli_time():
    return round(time.time() * 1000)
def is_m114_feedback(printeroutput):
    if(len(re.findall("(?:[^E][:])(\d{1,3}[.]\d{4})", printeroutput))==3):
        return True
    return False
def parse_coords(printeroutput):
    return list(map(float, re.findall("(?:[^E][:])(\d{1,3}[.]\d{4})",printeroutput)))
def get_human_coords(printeroutput):
    coordlist = parse_coords(printeroutput)
    return 'X:'+str(coordlist[0])+' Y:'+str(coordlist[1])+' Z:'+str(coordlist[2])

In [ ]:
##================WIDGET CONSTRUCTION================##
globalpadding='0px 0px 0px 0px'
globalmargin='0px 0px 0px 0px'
machineout = Output(
    layout={
            'width':'50%',
            'height':'100%',
            'border': '1px solid white',
            'overflow':'scroll',
            'object-fit':'cover',
            'object-position':'center',
            'margin':globalmargin,
            'padding':globalpadding
           }
)
terminalout = Output(
    layout={
            'width':'100%',
            'height':'100%',
            'border': '1px solid white',
            'overflow':'scroll',
            'object-fit':'cover',
            'object-position':'center',
            'margin':globalmargin,
            'padding':globalpadding
           }
)
userin = Text(
    layout={
            'width':'100%',
            'height':'5%',
            'border': '1px solid white',
            'description':str(CMDPROMPT),
            'placeholder':'Type a command here',
            'object-fit':'cover',
            'object-position':'center',
            'margin':globalmargin,
            'padding':globalpadding
           },
    value='',
    placeholder='Type a command here',
    disabled=False
)
positiontext = Text(
    layout={
            'width':'100%',
            'height':'100%',
            'border': '1px solid white',
            'description':'',
            'placeholder':'0.0',
            'object-fit':'cover',
            'object-position':'center',
            'margin':globalmargin,
            'padding':globalpadding
           },
    value='',
    placeholder='',
    disabled=True
)
##================BUTTON GRID================##
buttongrid = GridspecLayout(
    3,
    6,
    layout=Layout(
        width='100%',
        height='25%',
        grid_gap='2px 2px',
        object_fit='cover',
        object_position='center',
        margin=globalmargin,
        padding=globalpadding
    )
)
buttongrid[0, 0] = fanbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Disable Fan',
    icon='fan',
    layout=Layout(width='100%', height='100%')
)
buttongrid[2, 0] = motorbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Disable Motors',
    icon='power-off',
    layout=Layout(width='100%', height='100%')
)
buttongrid[1, 0] = leftbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog X-',
    icon='arrow-left',
    layout=Layout(width='100%', height='100%')
)
buttongrid[1, 2] = rightbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog X+',
    icon='arrow-right',
    layout=Layout(width='100%', height='100%')
)
buttongrid[0, 1] = forwardbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog Y+',
    icon='arrow-up',
    layout=Layout(width='100%', height='100%')
)
buttongrid[2, 1] = backbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog Y-',
    icon='arrow-down',
    layout=Layout(width='100%', height='100%')
)
buttongrid[1, 1] = homebutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Home',
    icon='home',
    layout=Layout(width='100%', height='100%')
)
buttongrid[0, 2] = probebutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Probe Position',
    icon='crosshairs',
    layout=Layout(width='100%', height='100%')
)
buttongrid[2, 2] = probegridbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Probe Grid',
    icon='border-all',
    layout=Layout(width='100%', height='100%')
)
buttongrid[1, 3] = zhomebutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Home Z',
    icon='arrow-to-top',
    layout=Layout(width='100%', height='100%')
)
buttongrid[1, 3] = zhomebutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Home Z',
    icon='arrow-to-top',
    layout=Layout(width='100%', height='100%')
)
buttongrid[0, 3] = upbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog Z+',
    icon='arrow-up',
    layout=Layout(width='100%', height='100%')
)
buttongrid[1, 3] = zhomebutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Home Z',
    icon='level-up',
    layout=Layout(width='100%', height='100%')
)
buttongrid[2, 3] = downbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog Z-',
    icon='arrow-down',
    layout=Layout(width='100%', height='100%')
)
buttongrid[0, 4] = gridstartbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Set Grid Start',
    icon='caret-left',
    layout=Layout(width='100%', height='100%')
)
buttongrid[2, 4] = gridendbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Set Grid End',
    icon='caret-right',
    layout=Layout(width='100%', height='100%')
)

buttongrid[0, 5] = jogslider = widgets.FloatLogSlider(
    value=10,
    base=10,
    min=-2, # max exponent of base
    max=2, # min exponent of base
    step=1, # exponent step
    orientation='horizontal',
    continuous_update=True,
    description='Jog Step',
    readout_format='.2f'

)
buttongrid[1, 5] = feedslider = widgets.FloatSlider(
    value=2500,
    min=100,
    max=5000,
    step=100,
    description='Feed Rate',
    continuous_update=True,
    orientation='horizontal',
    readout_format='1d',
)
buttongrid[2,5] = positionlabel = widgets.Text(
    value=str(currentpos),
    placeholder=str(currentpos),
    description='Position:',
    disabled=True
)

In [ ]:
##================SURFACE PLOT WIDGET================## 
surfaceplot = go.FigureWidget(
    data=[go.Surface(x=plotdata[0],y=plotdata[1],z=plotdata[2])],
    layout=go.Layout(
        title='Surface Plot',
        autosize=False,
        margin=dict(l=0, r=0, b=0, t=40),
        scene=dict(
            xaxis = dict(nticks=plotdata[0].size, range=[0,plotdata[0][-1]],autorange='reversed'),
            yaxis = dict(nticks=plotdata[1].size, range=[0,plotdata[1][-1]],autorange='reversed'),
            zaxis = dict(nticks=10, range=[-10,10],),
            aspectmode='manual',
            aspectratio=dict(
                x=plotdata[0][-1]/plotdata[1][-1],
                y=1,
                z=1
            )
        )
    )
)
##================WEBCAM SETUP================##
webcam = widgets.Image(value=stream.get_frame(), format='PNG', 
                       layout=Layout(
                           width='100%',
                           height='100%',
                           object_fit='cover',
                           object_position='center',
                           margin='-5px 0px 0px 0px',
                           padding=globalpadding
                       )
                      )
##========GENERAL UI LAYOUT========##
tabgroup = widgets.Tab(children=[terminalout, webcam, surfaceplot],
                       layout=Layout(
                           width='100%',
                           height='75%',
                           padding=globalpadding,
                           margin=globalmargin,
                           object_fit='cover',
                           object_position='center'
                       )
                      )
tabgroup.set_title(0, 'Terminal')
tabgroup.set_title(1, 'Webcam')
tabgroup.set_title(2, 'Surface')

rightside = VBox([tabgroup, buttongrid],
                 layout=Layout(
                     width='50%',
                     align_items='center',
                     padding=globalpadding,
                     margin=globalmargin,
                     object_fit='cover',
                     object_position='center'
                 )
                )

termgroup = HBox([machineout, rightside],
                 layout=Layout(
                     width='100%',
                     height='95%',
                     padding=globalpadding,
                     margin=globalmargin,
                     object_fit='cover',
                     object_position='center'
                 )
                )

controlgroup = VBox([termgroup, userin],
                    layout=Layout(
                        width='100%',
                        height='600px',
                        padding=globalpadding,
                        margin=globalmargin,
                       object_fit='cover',
                       object_position='center'
                    )
                   )

In [ ]:
##================MACRO DEFINITIONS================##
def show_help(b=None):
    """Shows this help"""
    terminalout.append_display_data('Available macros:')
    for macro in macros.items():
        terminalout.append_display_data(macro[0]+': '+macro[1].__doc__)
def update_surface_plot():
    global surfaceplot
    surfaceplot.update_layout(
        title='Surface Plot',
        autosize=False,
        margin=dict(l=0, r=0, b=0, t=40),
        scene=dict(
            xaxis = dict(nticks=plotdata[0].size, range=[0,plotdata[0][-1]],autorange='reversed'),
            yaxis = dict(nticks=plotdata[1].size, range=[0,plotdata[1][-1]],autorange='reversed'),
            zaxis = dict(nticks=10, range=[-10,10],),
            aspectmode='manual',
            aspectratio=dict(
                x=plotdata[0][-1]/plotdata[1][-1],
                y=1,
                z=1
            )
        )
    )
def probe_grid(b=None):
    """Returns array of probed coordinate"""
    global surfaceplot, plotdata, gridstart, gridend
    plotdata=create_plot_data()
    surfaceplot.data[0].x = plotdata[0]
    surfaceplot.data[0].y = plotdata[1]
    surfaceplot.data[0].z = plotdata[2]
    update_surface_plot()
    for x in plotdata[0]:
        for y in plotdata[1]:
            tasq.put(functools.partial(send_gcode, gcode='G00 F5000 '+'X'+str(x+gridstart[0])+' Y'+str(y+gridstart[1])))
            terminalout.append_display_data(str(x)+'|'+str(y))
def probe_point(b=None):
    """Returns the probed coordinate"""
    tasq.put(functools.partial(send_gcode, gcode='G60 S0'))
    tasq.put(functools.partial(send_gcode, gcode='M401')) #deploy probe
    tasq.put(functools.partial(send_gcode, gcode='G38.2 Z0')) #probe downwards
    tasq.put(functools.partial(send_gcode, gcode='M402')) #stow probe
    tasq.put(functools.partial(send_gcode, gcode='G61 Z S0')) #move to safe area
def home(b=None, axis=None):
    """Auto Home XYZ if axis=None"""
    gcode = 'G28'+axis if axis else 'G28' 
    tasq.put(functools.partial(send_gcode, gcode='G28'  ))
def deploy_probe(b=None):
    """deploy_probe the probe"""
    tasq.put(functools.partial(send_gcode, gcode='M401'))
def stow_probe(b=None):
    """stow_probe the probe"""
    tasq.put(functools.partial(send_gcode, gcode='M402'))
def enable_fan(b=None):
    """Set case fan speed to MAX"""
    tasq.put(functools.partial(send_gcode, gcode='M106 P2 S255'))
def disable_fan(b=None):
    """Disable all fans"""
    tasq.put(functools.partial(send_gcode, gcode='M106 P2 S0'))
def get_pos(b=None):
    """Report the current tool position"""
    tasq.put(functools.partial(send_gcode, gcode='M114'))
def disable_motors(b=None):
    """Disables all the motors"""
    tasq.put(functools.partial(send_gcode, gcode='M18'))
def set_relative(b=None):
    """Set all axes to relative"""
    tasq.put(functools.partial(send_gcode, gcode='G91'))
def set_absolute(b=None):
    """Set all axes to asbolute"""
    tasq.put(functools.partial(send_gcode, gcode='G90'))
def do_jog(b=None, direction=[0,0,0]):
    """Jogs machine"""
    tasq.put(functools.partial(send_gcode,
                               gcode='G91'))
    tasq.put(functools.partial(send_gcode,
                               gcode=str(
                                   'G00'+
                                   ' F'+str(feedslider.value)+
                                   ' X'+str(direction[0]*jogslider.value)+
                                   ' Y'+str(direction[1]*jogslider.value)+
                                   ' Z'+str(direction[2]*jogslider.value))))
    tasq.put(functools.partial(send_gcode,
                               gcode='G90'))
def queue_grid_start(b=None):
    tasq.put(functools.partial(send_gcode, gcode="M114"))
    tasq.put(functools.partial(set_grid_start))
def queue_grid_end(b=None):
    tasq.put(functools.partial(send_gcode, gcode="M114"))
    tasq.put(functools.partial(set_grid_end))
def set_grid_start(b=None):
    global gridstart, currentpos, got_ok
    gridstart=currentpos
    terminalout.append_display_data('GRID START: '+str(gridstart))
    got_ok=True
def set_grid_end(b=None):
    global gridend, currentpos, got_ok
    gridend=currentpos
    terminalout.append_display_data('GRID END: '+str(gridend))
    got_ok=True
##================MACRO DICTIONARY================##
macros = {
    'help':show_help,
    'probe':probe_point,
    'grid':probe_grid,
    'home':home,
    'deploy':deploy_probe,
    'stow':stow_probe,
    'fon':enable_fan,
    'foff':disable_fan,
    'pos':get_pos,
    'moff':disable_motors,
    'rel':set_relative,
    'abs':set_absolute
}

In [ ]:
##======================== BUTTON CALLBACKS ================##
userin.on_submit(handle_input)
leftbutton.on_click(functools.partial(do_jog, direction=[-1.0,0.0,0.0]))
rightbutton.on_click(functools.partial(do_jog, direction=[1.0,0.0,0.0]))
forwardbutton.on_click(functools.partial(do_jog, direction=[0.0,1.0,0.0]))
backbutton.on_click(functools.partial(do_jog, direction=[0.0,-1.0,0.0]))
upbutton.on_click(functools.partial(do_jog, direction=[0.0,0.0,1.0]))
downbutton.on_click(functools.partial(do_jog, direction=[0.0,0.0,-1.0]))
homebutton.on_click(functools.partial(home))
zhomebutton.on_click(functools.partial(home, axis='Z'))
probebutton.on_click(functools.partial(probe_point))
probegridbutton.on_click(probe_grid)
fanbutton.on_click(disable_fan)
motorbutton.on_click(disable_motors)
gridstartbutton.on_click(functools.partial(queue_grid_start))
gridendbutton.on_click(functools.partial(queue_grid_end))

In [ ]:
def _update():
    global got_ok
    got_ok=True
    while True:
        #printer output echo
        serialdata = read_serial()
        if serialdata != '':
            machineout.append_display_data('RECV: '+serialdata)
            if contains_ok(serialdata):
                got_ok=True
                #task.task_done()
            if is_m114_feedback(serialdata):
                terminalout.append_display_data('POSITION: '+get_human_coords(serialdata))
                global currentpos
                currentpos=parse_coords(serialdata)
                global positionlabel
                positionlabel.value = str(currentpos)
        #queue processing
        if got_ok:
            if not tasq.empty():
                got_ok=False
                task = tasq.get()
                #terminalout.append_display_data(str(task))
                task()
        time.sleep(0.01)

def _getstream(frameRate):
    while True:
        webcam.value = stream.get_frame()
        time.sleep(1.0/frameRate)


In [ ]:
myport=try_connect(1)

In [ ]:
myport.reset_input_buffer()
myport.reset_output_buffer()
machineout.clear_output()
terminalout.clear_output()

t_serialupdate = threading.Thread(target=_update, args=())
t_webcam = threading.Thread(target=_getstream, args=(15,))
t_serialupdate.start()
t_webcam.start()

display(controlgroup)

print('Enter your command in the box above. Press <ENTER> to send.')
print('Type "help" to see available shorcuts')

In [ ]:
print(currentpos)
print(gridstart)
print(gridend)

##### 

In [ ]:
print(plotdata[0])
print(plotdata[1])
print(plotdata[2])

In [ ]:
print(plotdata)